<p style="text-align: center;font-size: 40pt">Differential geometry</p>

In [1]:
%matplotlib widget
#%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from matplotlib import cm

import numpy as np
from scipy import spatial

%run ./scripts/helper_func.py
path = "{0}/lessons/transformations_2d/scripts/helper_func.py".format(get_root_path())
%run $path
path = "{0}/lessons/transformations_3d/scripts/helper_func.py".format(get_root_path())
%run $path
path = "{0}/common/scripts/style.py".format(get_root_path())
%run $path

# Overview 

Requirement
- [Uncertainty in 3D](2-lesson_uncertainty.ipynb)

Objectives of this lesson:

- introduce the concepts of geometric primitives
- give practical examples of statistical analysis of point clouds
- explain how to extract lines from noisy point clouds


Hidden custom latex commands here $ \curvearrowright$

----
[comment]: <> (General commands)
$\newcommand{\textcomma}{\quad\text{,}}$
$\newcommand{\textdot}{\quad\text{.}}$
$\newcommand{\vec}[1]{\overrightarrow{#1}}$
$\newcommand{\mat}[1]{\mathbf{#1}}$
$\newcommand{\frame}[1]{\mathcal{#1}}$
$\newcommand{\point}[2][]{{}^{#1}\mathbf{#2}}$
$\newcommand{\pointsym}[2][]{{}^{#1}\boldsymbol{#2}}$
$\newcommand{\matsym}[1]{\boldsymbol{#1}}$
$\newcommand{\real}{\mathbb{R}}$
$\newcommand{\bmat}[1]{\begin{bmatrix}#1\end{bmatrix}}$
$\newcommand{\F}[2][]{{}_{#2}^{#1}\mathscr{F}}$
$\newcommand{\Fmat}[2][]{{}_{#2}^{#1}\mat{F}}$
$\newcommand{\origin}[2][]{{}_{#2}^{#1}\mat{o}}$
$\newcommand{\T}[2][]{{}_{#2}^{#1}\mat{T}}$
$\newcommand{\t}[2][]{{}_{#2}^{#1}\mat{t}}$
$\newcommand{\R}[2][]{{}_{#2}^{#1}\mat{R}}$
$\newcommand{\f}{\vec{\mathscr{f}}}$
$\newcommand{\ax}[2][]{{}_{#2}^{#1}\vec{\mathscr{x}}}$
$\newcommand{\ay}[2][]{{}_{#2}^{#1}\vec{\mathscr{y}}}$
$\newcommand{\az}[2][]{{}_{#2}^{#1}\vec{\mathscr{z}}}$
$\newcommand{\aw}[2][]{{}_{#2}^{#1}\vec{\mathscr{w}}}$
$\newcommand{\axi}{\mathscr{x}}$
$\newcommand{\ayi}{\mathscr{y}}$
$\newcommand{\azi}{\mathscr{z}}$
$\newcommand{\awi}{\mathscr{w}}$
$\newcommand{\pointx}[2][]{{}^{#1}{#2}_{\axi}}$
$\newcommand{\pointy}[2][]{{}^{#1}{#2}_{\ayi}}$
$\newcommand{\pointz}[2][]{{}^{#1}{#2}_{\azi}}$
$\newcommand{\SO}[1]{\mathrm{SO}(#1)}$
$\newcommand{\var}{\text{Var}}$
----

# Geometric primitives

When only geometric information is available, there are still ways to extract some level of distinctness by using differential geometry.
We shortly introduce key concepts related to the use of geometric information.
In this work, the notion of a shape $\mathcal{S}$ is used as a representation of a generic object in the Euclidean space with a certain set of properties.
For example, those properties can be photometric, thermic, and semantic. 
Simple shapes, such as points, lines, quadrics, can be easily parametrized, but most of the shapes encountered in the a real environment are too complex to be completely synthesized with parameters.
To allow a certain representation of the world, a complex shape $\mathcal{S}$ can be approximated by a set of other shapes only if they can be expressed in the same frame of reference.


Sensors measuring depth produce such approximations by sampling the environment and producing a set of points. 
From smooth areas defined by those points, we present five types of primitives that can be extracted based on _differential geometry_: point, line, plane, curve and quadric. 
The first derivative group rely on normal vectors $\mat{n}$ (i.e., a vector orthogonal to a line or plane) and tangent vectors $\mat{t}$ (i.e. a vector parallel to a line or plane) to express the local area.
Since they are defined with respect to a point $\mat{p}$, normal and tangent vectors can be represented with a minimal set of two angles in polar coordinates.
Normal and tangent vectors can be seen as a dual representation.
The choice of using either one or the other is defined by the minimum information required to express a primitive.
In the case of a line in 3D, the normal vectors needs to define a plane perpendicular to that line. 
Therefore, only using the tangent vector is more convenient.
The same reasoning holds for using a normal vector for the surface.
The notion of direction also needs to be defined depending of the primitive.
It is reasonable to use unsigned direction in the case of tangents and signed direction in the case of normals where positive sign defined the outer surface of the shape.
The motivation behind this choice is that we want to keep track of which side of a surface we are measuring while moving.
In the 2D case, it is equivalent to represent a line by a tangent or a normal in term of the number of parameters.
However, it is usually assumed that the perceived 2D plane cuts perpendicular surfaces, so it makes more sense to use normal vector to also track the outer side of the line.
The next figure illustrates this choice of representation for the 2D case.
Viewpoints $\mat{v}_n$ (i.e., where the sensor was when a point $\mat{p}$ was measured on a surface $\mathcal{S}$) are used to determine the direction of the normal vectors $\mat{n}$, whereas no extra information is needed for a tangent vector $\mat{t}$ laying on a line that can be observed from both sides.

<p style="text-align:center">
    <img src="images/normals.jpg" width=30% />
    <img src="images/tangents.jpg" width=30.5% />
    <br>
    <b>Figure:</b> Difference between using normal vectors and tangent vectors to represent a 2D shape $\mathcal{S}$.
<em>Left</em>: The 2D line is known to be measured from a volume, which can only view from one side as illustrated with the black arrows $\mat{v}_1$ and $\mat{v}_2$. 
The direction of the surface make sense to be encoded in its representation leading to the choice of normal vectors $\mat{n}$ (dashed blue arrow).
<em>Right</em>: The same line can be observe from both direction leading to the selection of an undirected tangent vector $\mat{t}$ (dashed blue line).
</p>

As for the second derivative group, a curve is parametrized by a scalar representing curvature $\kappa$.  Curvature can be used to represent an osculating circle parallel to the tangent of a line. 
In the case of a quadric (i.e. a curved surface), it is parametrized by principal direction vectors $\mat{t}_\text{min}$, $\mat{t}_\text{max}$ and the principal curvature scalars $\kappa_\text{min}, \kappa_\text{max}$.
Those principal directions rely on a point $\mat{p}$ and on the normal vector $\mat{n}$, so they could be expressed only with one angle.
In other words, the principal directions are tangent vectors to the surface complementing the normal vector.
In the case of curves, the curvature is always positive as opposed to quadrics for which a positive $\kappa$ means that the surface bend in the same direction of the normal vector and vice-versa.
For simplicity in further computation, most of the publications use normalized 3D vectors to express normals and tangents leading to a larger set of parameters, as shown in the following table:

| Name   | <div style="width:150px">Parameters</div> | <div style="width:150px">Constraints</div> | Reference |
|:-----  |:--------   |:----------  |:------    |
|_Minimum parametrization:_
|Point   | $\mat{p} = \{x, y, z\}$       | $ \mat{p} \in \real^3$        | $\F{}$           |
|Tangent | $\mat{t} = \{\theta, \phi \}$ | $ \theta \in [ -\pi, \pi )$ | $\F{}$           |
|        |                               | $ \phi \in [ -\frac{\pi}{2}, \frac{\pi}{2} ] $||
|Normal  | $\mat{n} = \{\theta, \phi \}$ | $ \theta \in [ -\pi, \pi )$ | $\F{}$           |
|        |                               | $\phi \in [ -\frac{\pi}{2}, \frac{\pi}{2} ]$  ||
|Principal Directions | $\psi$   | $\psi \in [ -\pi, \pi )$    | $\{\mat{n}, \F{}\}$      |
|Curvature            | $\kappa$ | $\kappa \in \real_+$        | $\F{}$                   |
|Principal Curvatures | $\matsym{\kappa} = \{\kappa_\mathrm{min}, \kappa_\mathrm{max} \}$ | $\mat{\kappa} \in \real^2$ | $\{\mat{n}, \F{}\}$ |
|
|_Typical parametrization:_
|Point   | $\mat{p} = \{x, y, z\}$       | $\mat{p} \in \real^3$ | $ \F{}$  |
|Tangent | $\mat{t} = \{t_x, t_y, t_z\}$ | $|\mat{t}| = 1$     | $ \F{} $ |
|Normal  | $\mat{n} = \{n_x, n_y, n_z\}$ | $|\mat{n}| = 1$     | $ \F{} $ |
|Principal Directions | $\mat{\gamma} = \{t_\mathrm{min}, t_\mathrm{max}\}$ | $n \perp t_{min} \perp t_{max}$  | $\F{} $  |
|Curvature            | $\kappa$ | $\kappa \in \real_+$ | $\F{}$ |
|Principal Curvatures | $\mat{\kappa} = \{\kappa_\mathrm{min}, \kappa_\mathrm{max} \}$ | $\mat{\kappa} \in \real^2$ | $\{\mat{n}, \F{}\}$ |
<p style="text-align:center">
    <b>Table:</b> Comparison of different parametrizations used to represent geometric primitives.
</p>


Those parameters (i.e., point, tangent, normal, principal direction, curvature, and principal curvatures) bring us to a group of parametrized primitives (i.e., point, line, plane, curve and quadric), which can be helpful to approximate other complex 3D shapes. 
Those geometric primitives are listed in the next table with their characteristics.


|Primitives | <div style="width:150px">Parameters</div> | Derivative | Manifold | Bounded by | Nb Parameter |
|:------    |:--------   |:-------    |:-----    |:------|:---------    |
|Point   | $\mat{p}$                           | 0 | 0 | -           | 3(3) |
|Line    | $\mathcal{L} = \{ \mat{p}, \mat{t}\}$  | 1 | 1 | point       | 6(5) |
|Plane   | $\Phi = \{ \mat{p}, \mat{n}\}$      | 1 | 2 | line, curve | 6(5) |
|Curve   | $\mathcal{C} = \{ \mat{p}, \mat{t}, \mat{n}, \kappa \}$            | 2 | 1 | point       | 10(7) |
|Quadric | $\mathcal{Q} = \{ \mat{p}, \mat{n}, \mat{\gamma}, \mat{\kappa} \}$ | 2 | 2 | line, curve | 14(8) |
<p style="text-align:center">
    <b>Table:</b> Characteristics of primitives used for shape approximations. The number in parenthesis of the column <em>Nb Parameter</em> corresponds to the minimum number of parameters that can be used to express the same primitive.
</p>

A graphical representation of 1) shape considered as a 1D manifold and 2) shape considered as a 2D manifold are also showed in the two following figures.
In their current configuration, lines, planes, curves and quadrics are unbounded, which means that they can reach infinity on their unconstrained direction. 
One can constrain a primitive by using another primitive with a lower dimensionality. 
Thus, a plane can be bounded by a set of lines, a line can be bounded by a set of points, and so on. 


<p style="text-align:center">
    <img src="images/primitive1D.jpg" width=70% />
    <br>
    <b>Figure:</b> Representations (in dark blue) of a complex shape $\mathcal{S}$ (in light gray) approximated as a 1D manifold. 
    <em>Left</em>: no derivative (point). 
    <em>Middle</em>: first derivative (line). 
    <em>Right</em>: second derivative (curve).
</p>

<p style="text-align:center">
    <img src="images/primitive2D.jpg" width=70% />
    <br>
    <b>Figure:</b> Representations (in dark blue) of a complex shape $\mathcal{S}$ (in light gray) approximated as a 2D manifold. 
    <em>Left</em>: no derivative (point). 
    <em>Middle</em>: first derivative (plane). 
    <em>Right</em>: second derivative (quadric)
</p>

# Simple line reconstruction

Reconstructing geometric solely from noisy points requires statistical analysis.
We will apply the theory on covariance from the [last lesson](2-lesson_uncertainty.ipynb) to recover a line from a small point cloud.
Let's assume that we have a small point cloud that is part of a line.
We can find the vector tangent of to line by:
1. computing the covariance matrix of all points
1. computing the eigenvalues and eigenvectors of that covariance matrix
1. select the biggest eigenvalue and report its associated eigenvector as the tangent

In the case you are interested in the normal vector, you can take the eigenvector associated to the smallest eigenvalue instead.
The following code implement this algorithm.
From that code:
- can you find the different steps in the code?
- can you find which red arrows of the graph is the normal vector? the tangent?

In [6]:
def draw_cov(ax, data, scale=1):
    mu = np.mean(data, axis=1)
    error = data.T - mu
    Sigma = 1/n * (error.T @ error)
    lambdas, R = np.linalg.eig(Sigma)
    
    # plot
    cercle = build_cercle_xy(scale, 100)[:2]
    elipse = Sigma @ cercle
    ax.plot(elipse[0], elipse[1], color='tab:red')
    draw_frame(ax, x = 10*R[:,0], y = 10*R[:,1], color="tab:red")

# build data set
n = 100
lim = 10
scale = 1/3.
data = np.random.uniform(low=[-lim*scale, -lim], high=[lim*scale, lim], size=(n,2)).T
R = rot_2d(0.6)
data = R @ data

#--------------------------------
# plotting
if 'fig' in globals():
    plt.close(fig)
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax.scatter(data[0], data[1])

draw_cov(ax, data, scale=0.5)
ax.axis('equal');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Locality and spatial search

Point cloud from a room would have multiple lines and planes, not a single one.
If you would extract the covariance from a complete point cloud, it wouldn't be really descriptive.
This method of extracting geometric information from the covariance works only if you have access to a small subset of the point that are locally coherent (e.g., all the points are on the same plane).
The technique use in that case is to only evaluate the point surrounding a small area of the complete point cloud.
For each point in a point cloud, we can search for its $k$ nearest neighbors and only compute the covariance for those points.
If you think about it, this operation is quite expensive in terms of computation time.
For all points, you need to compute the distance to all other points, sort all the distances and select the $k$ points with the smallest distances.
Without going into too much detail, a **kd-tree** is an algorithm which first structure a point cloud as a tree and then execute a spacial search in that tree to find the $k$ nearest neighbors of a query point.
A kd-tree allows you to considerable reduce the computation time of a spatial search.
There are two main types of spatial search:
1. requesting the $k$ nearest points;
2. fixing a radius, and requesting all points in that radius.

The following code let you play with the kd-tree implemented in `numpy` on two generated data sets.
With the combination of the code and the graph, investigate
- what happens when you change the coordinate of the query point `query_pt`?
- what is the impact of making a radius search versus a $k$ nearest points search by using the variable `is_query_ball`?

In [8]:
# change coordinates
query_pt = np.array([0,10])

# change to switch between knn (False) and ball (True) methods
is_query_ball = False # or True

# search parameters
k_nn = 100 # number of nearest-neighbor
radius = 5.

# data set parameters
n = 1000
lim = 10
lim_mask = 9

# build data sets
data1 = np.random.uniform(low=[-lim, -lim], high=[lim, lim], size=(n,2))

mask = (data1[:,0] > lim_mask) + (data1[:,0] < -lim_mask) + (data1[:,1] > lim_mask) + (data1[:,1] < -lim_mask)
data2 = data1[mask]

# build kdtree
tree1 = spatial.KDTree(data1)
tree2 = spatial.KDTree(data2)

# search in the trees
if is_query_ball:
    index1 = tree1.query_ball_point(query_pt, radius)
    index2 = tree2.query_ball_point(query_pt, radius)
else:
    dist1, index1 = tree1.query(query_pt, k=k_nn)
    dist2, index2 = tree2.query(query_pt, k=k_nn)

# mark selection
selected1 = np.zeros(len(data1))
selected2 = np.zeros(len(data2))
selected1[index1] = 1
selected2[index2] = 1

#--------------------------------
# plotting
if 'fig' in globals():
    plt.close(fig)
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(121)
ax.scatter(data1[:,0], data1[:,1], c=selected1)
ax.scatter(query_pt[0], query_pt[1], c='red', marker='*', s=80)

ax = fig.add_subplot(122)
ax.scatter(data2[:,0], data2[:,1], c=selected2)
ax.scatter(query_pt[0], query_pt[1], c='red', marker='*', s=80);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Extract surface tangents

Finally, we can put all the parts together to extract an estimate of the tangent vector for all points in a point cloud.
The following code uses a radius search and a modified version of the eigendecomposition function of `numpy`.
The function `sorted_eig()` extract the eigenvalues and eigenvector, but sort them before returning them, so that we are sure that $\lambda_1 < \lambda_2$.
The first row of graphs show the ratio of eigenvalues $\frac{\lambda_1}{\lambda_2}$ for you to understand that the lower this ratio is, the flatter is the covariance meaning that the direction should be well estimated.
The second row of graphs shows the extracted tangent vector.
Note that the first data set is given as an example of an area where it doesn't make sense to fit a line.

In [11]:
def sorted_eig(A):
    eigenValues, eigenVectors = np.linalg.eig(A)
    idx = np.argsort(eigenValues)
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return (eigenValues, eigenVectors)

def extract_struct(tree, data, radius):
    min_eigenvalue = np.zeros(len(data))
    max_eigenvector = np.zeros_like(data)
    
    for i, query_pt in enumerate(data):
        # nn search
        index = tree.query_ball_point(query_pt, radius)
        n = len(index)
        if n > 0:
            neighbors = data[index,:]

            # statistics
            mu = np.mean(neighbors, axis=0)
            error = neighbors - mu
            Sigma = 1/n * (error.T @ error)
            lambdas, R = sorted_eig(Sigma)
            min_eigenvalue[i] = lambdas[0]/lambdas[1]
            max_eigenvector[i,:] = R[:,-1]
            
    return min_eigenvalue, max_eigenvector

# search parameter
radius = 5.

# data sets parameters
n = 1000
lim = 10
lim_mask = 9

# build data sets
data1 = np.random.uniform(low=[-lim, -lim], high=[lim, lim], size=(n,2))

mask = (data1[:,0] > lim_mask) + (data1[:,0] < -lim_mask) + (data1[:,1] > lim_mask) + (data1[:,1] < -lim_mask)
data2 = data1[mask]

# build kdtree
tree1 = spatial.KDTree(data1)
tree2 = spatial.KDTree(data2)

# search for all points
ratio1, max_eigenvector1 = extract_struct(tree1, data1, radius=radius)
ratio2, max_eigenvector2 = extract_struct(tree2, data2, radius=radius)

#--------------------------------
# plotting
if 'fig' in globals():
    plt.close(fig)
fig = plt.figure(figsize=(8,8))

vmin = np.min(np.hstack([ratio1, ratio2]))
vmax = np.max(np.hstack([ratio1, ratio2]))

ax = fig.add_subplot(221)
ax.set_title("Data set 1")
ax.scatter(data1[:,0], data1[:,1], c=ratio1, vmin=vmin, vmax=vmax)

ax = fig.add_subplot(223)
ax.quiver(data1[:,0], data1[:,1], max_eigenvector1[:,0], max_eigenvector1[:,1], color="white")

ax = fig.add_subplot(222)
ax.set_title("Data set 2")
im = ax.scatter(data2[:,0], data2[:,1], c=ratio2, vmin=vmin, vmax=vmax)
cbar = fig.colorbar(im)
cbar.set_label('Eigenvalue ratio')

ax = fig.add_subplot(224)
ax.quiver(data2[:,0], data2[:,1], max_eigenvector2[:,0], max_eigenvector2[:,1], color="white");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Conclusion

Actions:
- run the code in your notebook and play with the parameters to understand the behavior of the computation show as examples;
- modify the markdown by adding your own notes using `> my notes` to help you review the material later; and
- complete the table [Glossary](#Glossary) and add your own definitions.

Next module:
- [Point cloud registration](../registration/0-overview.ipynb)

## Glossary

| English             | Français            | Definition |
|-----------          |------------         |:--------   |
| kd-tree             | arbre kd            |            |
| eigenvectors        | vecteurs propres    |            |
| eigenvalues         | valeurs propres     |            |
| curvature           | courbure            |            |
| ...                 |                     |            |